In [1]:
import pandas as pd

In [2]:
# CO₂ Total Emissions einlesen
df = pd.read_csv('../data/raw/co2_total.csv')
display(df.head())

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Nur Jahre ab 1950
df = df[df['Year'] >= 1950]

# Länder ohne ISO-Code ausschließen
df = df[df['ISO 3166-1 alpha-3'].notna()]

# Zeilen ohne 'Total'-Wert entfernen
df = df[df['Total'].notna()]

# Index zurücksetzen
df.reset_index(drop=True, inplace=True)

In [4]:
df.rename(columns={
    'ISO 3166-1 alpha-3': 'ISO3',
    'Per Capita': 'PerCapita'
}, inplace=True)

In [6]:
# In processed-Ordner speichern
df.to_csv('../data/processed/co2_cleaned_total.csv', index=False)
print("Datei gespeichert: co2_cleaned_total.csv")

Datei gespeichert: co2_cleaned_total.csv
